# Revised case normalization for Hirslanden Zurich 2018

Notes:
- There is 'n.ü.' in the duration_of_stay, at the moment replace them with nan and deleted
- The case in row 96 can not be grouped, we deleted it at the moment. 



In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group, format_for_grouper, format_for_grouper_one_case, group_batch_group_cases
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(
        os.path.join(ROOT_DIR, 'raw_data/HI-Zurich.xlsx'),
        'Hirslanden Zurich',
         '2018',
        'Änderungen_Hirslanden_2018')

print(file_info)


FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI-Zurich.xlsx', hospital_name_db='Hirslanden Zurich', year='2018', sheet='Änderungen_Hirslanden_2018')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-10 14:23:38.329 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:35 - Read 120 cases for Hirslanden Zurich 2018
2022-11-10 14:23:38.335 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:60 - TYPES:
datum                                                string
unnamed: 1                                           string
case_id                                              object
patient_id                                           object
gender                                               string
age_years                                             int64
bfs_code                                             string
duration_of_stay                                      int64
pflegetage neu                                       string
pccl                                                  int64
pccl neu                                             string
old_pd                                               string
primary

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,0041622923,D57E1A8A84BA5715,M,76,19,41622923,C20,C20,M200,3,G17Z,[E440],[E441],[],[]
1,0041628878,1EAE82BF47B46A42,M,84,5,41628878,C9200,C9200,M100,3,R60E,[B909],[],[],[]
2,0041634840,69B604C8EDC65FDA,M,66,17,41634840,T818,T818,M100,3,X06B,"[E1173, T842, I5013]","[E1121, E1151, E1141]",[],[]
3,0041643502,D3F33C4023F17E75,W,72,14,41643502,M161,M161,M200,3,I08C,[D62],[],[],[]
4,0041657438,B26B73105A4B1FE3,M,66,8,41657438,K835,K835,M200,3,H01B,"[D1803, I420]","[D134, Z867]",[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0044091599,FF1CA55986D937DB,M,75,7,44091599,C180,C180,M100,3,G60B,[E43],[],[],[]
116,0044093471,F44CB526CF8A238E,W,55,6,44093471,T827,T827,M100,3,F75D,[D592],[D648],[],[]
117,0044094461,9FDAF84F7F440BFA,W,82,7,44094461,N10,N10,M100,3,L63D,"[U804, B966]",[],[],[]
118,0044098568,CDD9C616F85275E8,W,64,12,44098568,I351,I351,M200,3,F06B,"[E871, I5001, I652]",[],[],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases.head()

2022-11-10 14:23:40.507 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:90 - Read 18276 rows from the DB, for the hospital 'Hirslanden Zurich' in 2018


,aimedic_id,case_id,primary_diagnosis,secondary_diagnoses,primary_procedure,secondary_procedures,gender,age_years,age_days,gestation_age,duration_of_stay,ventilation_hours,grouper_admission_type,admission_date,admission_weight,grouper_discharge_type,discharge_date
0,156117,0044049518,C481,"[J90, E780, M8189, Z850, Z904, ...]",3481:R:20180814,"[99B711::20180814, 4143::20180814, 346X23::20180814, 502512::20180814, 009A14::20180814, ...]",W,85,0,0,11,0,01,2018-08-13,0,00,2018-08-24
1,156193,0044059102,K449,"[K210, K660, K861, J459, Z867, ...]",446720::20180924,[5451::20180924],M,64,0,0,4,0,01,2018-09-23,0,00,2018-09-27
2,156276,0044016672,T845,"[Y828, T813, Y849, D62, L8914, ...]",815121:L:20180515,"[86531E:L:20180526, 009910::20180526, 990410::20180516, 874199::20180514]",W,87,0,0,26,0,01,2018-05-14,0,00,2018-06-09
3,156455,0041634840,T818,"[Y849, T814, N083, N183, I792, ...]",866B2C:L:20180112,"[009910::20180112, 862A1E:L:20180112, 86531E:L:20180112, 874199::20180106, 883840::20180115]",M,66,0,0,17,0,01,2018-01-06,0,00,2018-01-23
4,156530,0044047360,C250,"[C787, I2513, Z955, E788, E038, ...]",893909::20180730,[],M,73,0,0,11,0,01,2018-07-30,0,00,2018-08-10


In [5]:
if unmatched.shape[0] > 0:
    unmatched

In [6]:
revised_cases.loc[96:97]

,aimedic_id,case_id,primary_diagnosis,secondary_diagnoses,primary_procedure,secondary_procedures,gender,age_years,age_days,gestation_age,duration_of_stay,ventilation_hours,grouper_admission_type,admission_date,admission_weight,grouper_discharge_type,discharge_date
96,170750,0041703740,C20,"[N328, D6833, R31, Y579, N130, ...]",574912::20180306,"[009910::20180306, 570X12::20180306, 5732::20180306, 598X11:R:20180306]",M,74,0,0,4,0,01,2018-03-05,0,00,2018-03-09
97,170845,0044094461,N10,"[B962, N1791, N189, C9110, D508, ...]",992218::20181130,"[990410::20181201, 890A11::20181130]",W,82,0,0,7,0,01,2018-11-30,0,00,2018-12-07


In [7]:
# The row 96 can not be grouped, we deleted it at the moment
revised_cases.drop(96, inplace=True)

In [8]:
revised_cases.shape

(115, 17)

In [9]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

2022-11-10 14:23:41.304 | INFO     | src.revised_case_normalization.notebook_functions.group:group:83 - Grouping 115 cases ...
2022-11-10 14:23:42.486 | SUCCESS  | src.revised_case_normalization.notebook_functions.group:group:88 - Grouped 115 cases into: 115 revisions, 1273 diagnoses rows, 700 procedure rows


In [10]:
revisions_update

,aimedic_id,drg,drg_cost_weight,effective_cost_weight,pccl,revision_date
0,156117,G12A,3.38,3.38,4,2022-12-31
1,156193,G19A,2.71,2.71,4,2022-12-31
2,156276,I03A,4.26,4.26,4,2022-12-31
3,156455,X06A,2.83,2.83,4,2022-12-31
4,156530,H61A,1.35,1.35,4,2022-12-31
...,...,...,...,...,...,...
110,173155,I46A,2.61,2.61,4,2022-12-31
111,173502,Q61C,1.19,1.19,4,2022-12-31
112,173648,L63D,0.65,0.65,3,2022-12-31
113,174109,I46A,2.61,2.61,4,2022-12-31


In [11]:
update_db(revisions_update, diagnoses_update, procedures_update)

2022-11-10 14:23:42.504 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:249 - Trying to insert 115 cases into the 'Revisions' table ...
2022-11-10 14:23:43.294 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:275 - Deleted 115 rows from the "Revisions" table, which is about to be updated
2022-11-10 14:23:43.358 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:286 - Inserted 115 cases into the 'Revisions' table
2022-11-10 14:23:43.360 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:298 - Trying to insert 1273 rows into the 'Diagnoses' table ...
2022-11-10 14:23:45.943 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:336 - Inserted 1273 rows into the 'Diagnoses' table
2022-11-10 14:23:45.947 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:347 - Trying to insert 700 rows into the 'Procedure